In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import random
import numpy as np
import torch

random.seed(0)
np.random.seed(0)
_ = torch.manual_seed(0)
# _ = torch.cuda.manual_seed(42) # only if using GPU

In [4]:
import numpy as np
from tensorflow import keras

trained_models = np.load('trained_models.npy', allow_pickle=True)

In [171]:
from data_utils import create_dataset

model_tf = trained_models[5][1]

n_test = 1000
images, labels = create_dataset(n_test)
coefficients = np.array([0.4, 0.4, 0.4])

X_tf = images.reshape((-1, 28, 28, 1))
y_tf = np.matmul(labels, coefficients) > 0.6

predictions = model_tf.predict(X_tf)
accuracy = predictions.squeeze().round() == y_tf
print(f'Accuracy: {accuracy.mean():.4f}')

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
Accuracy: 0.9170


In [7]:
# self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1) # 28 x 28 x 1 -> 28 x 28 x 16
# self.relu1 = nn.ReLU()
# self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # 28 x 28 x 16 -> 14 x 14 x 16
# self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1) # 14 x 14 x 16 -> 14 x 14 x 32
# self.relu2 = nn.ReLU()
# self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # 14 x 14 x 32 -> 7 x 7 x 32
# self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1) # 7 x 7 x 32 -> 7 x 7 x 64
# self.relu3 = nn.ReLU()
# self.flatten = nn.Flatten()
# self.fc1 = nn.Linear(64 * 7 * 7, 3) # (7 x 7 x 64 = 3136) -> 3
# self.relu4 = nn.ReLU()
# self.fc2 = nn.Linear(3, 1) # 3 -> 1
# self.sigmoid = nn.Sigmoid()

In [9]:
model_tf.summary()

Model: "functional_200"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ keras_tensor_1800 (InputLayer)  │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer1 (Conv2D)                 │ (None, 28, 28, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_400               │ (None, 14, 14, 16)     │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer2 (Conv2D)                 │ (None, 14, 14, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_401               │ (None, 7, 7, 32)       │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer3 (Conv2D)                 │ (None, 7, 7, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_200 (Flatten)           │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 3)              │         9,411 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_200 (Dense)               │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 98,135 (383.34 KB)

 Trainable params: 32,711 (127.78 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 65,424 (255.57 KB)

In [14]:
from model_utils import PyTorchCNN

[p.numel() for p in PyTorchCNN().parameters()]

[144, 16, 4608, 32, 18432, 64, 9408, 3, 3, 1]

In [15]:
model_tf.layers[8].bias

<KerasVariable shape=(1,), dtype=float32, path=dense_200/bias>

In [129]:
from model_utils import PyTorchCNN

def keras_to_pytorch_cnn(keras_model):
    pytorch_model = PyTorchCNN()

    # Copy weights
    pytorch_model.conv1.weight.data = torch.FloatTensor(keras_model.get_layer('layer1').get_weights()[0].transpose(3, 2, 0, 1))
    pytorch_model.conv1.bias.data = torch.FloatTensor(keras_model.get_layer('layer1').get_weights()[1])

    pytorch_model.conv2.weight.data = torch.FloatTensor(keras_model.get_layer('layer2').get_weights()[0].transpose(3, 2, 0, 1))
    pytorch_model.conv2.bias.data = torch.FloatTensor(keras_model.get_layer('layer2').get_weights()[1])

    pytorch_model.conv3.weight.data = torch.FloatTensor(keras_model.get_layer('layer3').get_weights()[0].transpose(3, 2, 0, 1))
    pytorch_model.conv3.bias.data = torch.FloatTensor(keras_model.get_layer('layer3').get_weights()[1])

    pytorch_model.fc1.weight.data = torch.FloatTensor(keras_model.get_layer('dense1').get_weights()[0].T)
    pytorch_model.fc1.bias.data = torch.FloatTensor(keras_model.get_layer('dense1').get_weights()[1])

    pytorch_model.fc2.weight.data = torch.FloatTensor(keras_model.get_layer('dense_200').get_weights()[0].T)
    pytorch_model.fc2.bias.data = torch.FloatTensor(keras_model.get_layer('dense_200').get_weights()[1])

    return pytorch_model

In [140]:
import numpy as np

labels = np.load('labels.npy')
images = np.load('images.npy')
labels.shape, images.shape

((50000, 3), (50000, 28, 28))

In [177]:
from model_utils import evaluate, PyTorchCNN

# model_pt = keras_to_pytorch_cnn(model_tf)
model_pt = PyTorchCNN()
model_pt.load_state_dict(torch.load('pytorch_models/cnn_model_3.pth'))
coefficients = np.array([0.5, 0.4, 0.3])

# batch, channel, height, width
X = torch.tensor(images.reshape((-1, 1, 28, 28))).float()
y = torch.tensor(np.matmul(labels, coefficients) > 0.6).float()

# evaluate model
accuracy = evaluate(model_pt, X, y, batch_size=1024)
print(f'Accuracy: {accuracy:.4f}')

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  9.25it/s]

Accuracy: 0.9680


In [149]:
coef_1 = np.array([0.4, 0.4, 0.4])
coef_2 = np.array([0.3, 0.4, 0.5])

label_1 = np.matmul(labels, coef_1) > 0.6
# label_1_1 = np.sum(labels * coef_1, axis=1) > 0.6
label_2 = np.matmul(labels, coef_2) > 0.6

(label_1 == label_2).sum() / len(label_1)

0.92966

In [88]:
X[0, 0, 10, :].numpy()

array([1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.49, 0.49,
       0.49, 0.49, 0.49, 0.49, 0.49, 0.49, 0.49, 0.49, 0.49, 1.  , 1.  ,
       1.  , 1.  , 1.  , 1.  , 1.  , 1.  ], dtype=float32)

In [87]:
X_tf[0, 10, :, 0]

array([1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.49, 0.49,
       0.49, 0.49, 0.49, 0.49, 0.49, 0.49, 0.49, 0.49, 0.49, 1.  , 1.  ,
       1.  , 1.  , 1.  , 1.  , 1.  , 1.  ])

In [89]:
model_tf.get_layer('layer1').get_weights()[0][:, :, 0, 0]

array([[-0.27261996, -0.02060561,  0.01920205],
       [-0.2456749 ,  0.12463011, -0.1506281 ],
       [ 0.00114958, -0.00217168,  0.16942786]], dtype=float32)

In [90]:
model_pt.conv1.weight.data[0, 0, :, :]

tensor([[-0.2726, -0.0206,  0.0192],
        [-0.2457,  0.1246, -0.1506],
        [ 0.0011, -0.0022,  0.1694]])

In [20]:
print(X.shape, X_tf.shape)

model_tf.layers[1](X_tf).shape, model_pt.conv1(X).shape

torch.Size([1000, 1, 28, 28]) (1000, 28, 28, 1)


(TensorShape([1000, 28, 28, 16]), torch.Size([1000, 16, 28, 28]))

In [128]:
l1_tf = model_tf.layers[1](X_tf)
l1_pt = model_pt.relu1(model_pt.conv1(X))

torch.allclose(
    torch.FloatTensor(l1_tf.numpy().transpose(0, 3, 1, 2))[0, :, :, :],
    l1_pt[0, :, :, :],
    atol=1e-5
)

True

True

In [111]:
l1_tf.numpy().transpose(0, 3, 1, 2)[0, :, 1, 1]

array([0.        , 0.0937631 , 0.0341211 , 0.        , 0.02384765,
       0.02646165, 0.02025708, 0.04957731, 0.        , 0.12598737,
       0.16461095, 0.        , 0.        , 0.24812466, 0.10196638,
       0.        ], dtype=float32)

In [112]:
l1_pt.detach().numpy()[0, :, 1, 1]

array([0.        , 0.09376314, 0.03412114, 0.        , 0.02384765,
       0.02646163, 0.02025708, 0.04957734, 0.        , 0.1259874 ,
       0.16461095, 0.        , 0.        , 0.2481246 , 0.10196638,
       0.        ], dtype=float32)

In [65]:
option_a = np.expand_dims(model_tf.get_layer('layer1').get_weights()[0].squeeze().transpose(2, 0, 1), 1) 
option_b = model_tf.get_layer('layer1').get_weights()[0].reshape(PyTorchCNN().conv1.weight.shape)
option_c = model_tf.get_layer('layer1').get_weights()[0].transpose(3, 2, 0, 1)
option_d = model_tf.get_layer('layer1').get_weights()[0].transpose(3, 2, 1, 0)

option_a.shape, option_b.shape, option_c.shape, option_d.shape

((16, 1, 3, 3), (16, 1, 3, 3), (16, 1, 3, 3), (16, 1, 3, 3))

In [70]:
np.all(option_d == option_b)

False

In [42]:
PyTorchCNN().conv1.weight.shape

torch.Size([16, 1, 3, 3])

In [40]:
np.all(model_tf.get_layer('layer1').get_weights()[0] == model_pt.conv1.weight.data.numpy().transpose(2, 3, 1, 0))

True

In [62]:
model_tf.count_params()

32711

In [113]:
PyTorchCNN().conv1.weight.shape

torch.Size([16, 1, 3, 3])

In [112]:
model_tf.get_layer('layer1').get_weights()[0].shape

(3, 3, 1, 16)

In [39]:
model.fc1.weight.shape

torch.Size([3, 3136])

In [43]:
model_tf.get_layer('dense1').get_weights()[0].shape

(3136, 3)

In [2]:
import os
import numpy as np
import pandas as pd

VARIABLES = {
    '[0]': 'Circularity',
    '[1]': 'Color',
    '[2]': 'Area',
    '[0, 1]': 'Circularity, Color',
    '[0, 2]': 'Circularity, Area',
    '[1, 2]': 'Color, Area',
    '[0, 1, 2]': 'Circularity, Color, Area'
}
COEFFS = ['Equal (0.4, 0.4, 0.4)', 'Prefer Area (0.3, 0.4, 0.5)', 'Prefer Circularity (0.5, 0.4, 0.3)']

LAYERS = {
    'conv1': 'conv1<br>(28 x 28 x 16)',
    'conv2': 'conv2<br>(14 x 14 x 32)',
    'conv3': 'conv3<br>(7 x 7 x 64)',
    'fc1': 'fc1 (3)'
}

dfs = []
sorted_results = sorted(os.listdir('results'), key=lambda x: int(x.split('_')[0]))
for result in sorted_results:
    print('Loading:', result)
    dfs.append(pd.read_csv(f'results/{result}'))

df = pd.concat(dfs).reset_index(drop=True)
# add model index (each model has 4 layers x 7 variables = 28 rows)
df['model index'] = np.arange(len(df)) // 28

df['training objective'] = df['model index'].apply(lambda x: COEFFS[x % 3])

df['variables localized'] = df['variables'].map(VARIABLES)

df['layer'] = df['layer'].map(LAYERS)

df = df[['training objective', 'layer', 'variables localized', 'accuracy', 'model index']].rename(columns={
    'accuracy': 'interchange intervention accuracy'
})
df.to_csv('results.csv', index=False)

Loading: 0_30.csv
Loading: 30_60.csv
Loading: 60_90.csv
Loading: 90_120.csv
Loading: 120_150.csv
Loading: 150_180.csv
Loading: 180_210.csv
Loading: 210_240.csv
Loading: 240_270.csv
Loading: 270_300.csv


In [8]:
import plotly.express as px

category_orders = {
    'layer': list(LAYERS.values()),
    'variables localized': list(VARIABLES.values())
}

fig = px.box(df, y='interchange intervention accuracy', color='training objective', x='layer', facet_col='variables localized', category_orders=category_orders)
fig.show()

In [20]:
plot_df = df.groupby(['training objective', 'variables localized', 'layer'])['interchange intervention accuracy'].mean().reset_index()
plot_df['e'] = 1.96 * df.groupby(['training objective', 'variables localized', 'layer'])['interchange intervention accuracy'].sem().reset_index()['interchange intervention accuracy']

fig = px.scatter(plot_df, x='layer', y='interchange intervention accuracy', color='training objective', facet_col='variables localized', category_orders=category_orders, error_y='e')
fig.show()